In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
test_articles = pd.read_csv('../dataset/test_articles_ourdata.csv', delimiter=',')

In [ ]:
test_articles.info()

In [ ]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_ourdata.csv', delimiter=',')

In [ ]:
fixed_index.info()

In [ ]:
corpus = []

for i, row in tqdm(test_articles.iterrows()):
    
    corpus.append(row['article_title'])
    corpus.append(str(row['article_meta_description']))
    corpus.append(str(row['article_keywords']))
    
for i, row in fixed_index.iterrows():
    
    corpus.append(row['table_page_title'])
    corpus.append(str(row['table_page_main_passage']))
    corpus.append(str(row['table_page_keywords']))

vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

In [ ]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [ ]:
test_articles.info()

In [ ]:
fixed_index.info()

In [ ]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_id']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])
        article_keywords_text = str(row['article_keywords'])

        #embedding and model variables
        article_title1 = []
        article_title2 = []
        article_title3 = []
        title_table1 = []
        title_table2 = []
        title_table3 = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_id']]
        
        tfidf_vector = vectorizer.transform([article_title_text])
        article_title1.append(tfidf_vector[0].toarray()[0])
        
        tfidf_vector = vectorizer.transform([article_meta_description_text])
        article_title2.append(tfidf_vector[0].toarray()[0])
        
        tfidf_vector = vectorizer.transform([article_keywords_text])
        article_title3.append(tfidf_vector[0].toarray()[0])

        #creating embedding 
        for i, row in return_index.iterrows():
            
            tfidf_vector = vectorizer.transform([str(row['table_page_title'])])
            title_table1.append(tfidf_vector[0].toarray()[0])
            
            tfidf_vector = vectorizer.transform([str(row['table_page_main_passage'])])
            title_table2.append(tfidf_vector[0].toarray()[0])
            
            tfidf_vector = vectorizer.transform([str(row['table_page_keywords'])])
            title_table3.append(tfidf_vector[0].toarray()[0])
        
        distance_vector1 = pairwise_distances(article_title1, title_table1, metric='cosine')
        distance_vector2 = pairwise_distances(article_title2, title_table2, metric='cosine')
        distance_vector3 = pairwise_distances(article_title3, title_table3, metric='cosine')
        
        final_vector = (np.array(distance_vector1) + np.array(distance_vector2) + np.array(distance_vector3)) / 3.0
        
        #creating the final dataframe
        for i in range(0,len(final_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],final_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values
    
#         print("")
#         print("query:" +article_title_text)
#         print("")
#         print(draw_tables_socres)

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("TF-IDF METHOD")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")